# Import Libraries
Import necessary libraries including NumPy, Matplotlib, scikit-learn for data, pickle for model saving/loading, networkx for visualization, and tqdm for progress tracking.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Activation Functions Implementation
Implement activation function classes including the base Activation class, Linear, ReLU, Sigmoid, Tanh, and Softmax, each with forward and derivative methods.

# Loss Functions Implementation
Implement loss function classes including the base Loss class, MSE (Mean Squared Error), BinaryCrossEntropy, and CategoricalCrossEntropy, each with forward and derivative methods.

# Weight Initializers Implementation
Implement weight initializer classes including the base Initializer class, ZeroInitializer, UniformInitializer, and NormalInitializer.

# Layer Implementation
Implement the Layer class with forward and backward propagation methods, weight initialization, and gradient updates.

# FFNN Model Implementation
Implement the Feedforward Neural Network (FFNN) class with methods for creating the network, forward and backward propagation, weight updates, training, and prediction.

# Training and Evaluation on Dataset
Load the dataset, preprocess it for classification, and train the FFNN model with appropriate hyperparameters. Evaluate model performance using accuracy metrics.

In [ ]:
from activation import Linear, ReLU, Sigmoid, Tanh, Softmax, LeakyReLU, ELU
from loss import MeanSquaredError, BinaryCrossEntropy, CategoricalCrossEntropy
from initialization import ZeroInitialization, UniformInitialization, NormalInitialization, XavierInitialization, HeInitialization
from model import FFNN

### Load Dataset & Data Preparation

In [ ]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Label butuh di-encode dengan One Hot
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

In [ ]:
layer_sizes = [784, 128, 32, 10]  # Fitur input ada 784

activations = [
    LeakyReLU(),
    LeakyReLU(), 
    Softmax()     
]

loss_function = CategoricalCrossEntropy()

initializations = [
    HeInitialization(seed=42),  
    XavierInitialization(seed=42),  
    HeInitialization(seed=42)   
]

model = FFNN(
    layer_sizes=layer_sizes,
    activations=activations,
    loss=loss_function,
    initializations=initializations
)

history = model.train(
    x_train=X_train,
    y_train=y_train,
    batch_size=32,
    learning_rate=0.01,
    epochs=20,
    x_y_val=(X_test, y_test),
    verbose=1
)

# plot loss
plt.figure(figsize=(10, 6))
plt.plot(history['train_loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# prediksi
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred_classes == y_true_classes)
print(f"Test accuracy: {accuracy:.4f}")

# Model Visualization
Implement visualization of model architecture, training history, weight distributions, and gradient distributions using matplotlib and networkx.

In [ ]:
# # Visualisasi arsitektur model FFNN
# model.plot_model()

# # Grafik History Training (Epoch terhadap Loss)
# plt.figure(figsize=(10, 6))
# plt.plot(history['train_loss'], label='Training Loss')
# plt.plot(history['val_loss'], label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.show()

# Distribusi Bobot di tiap layer
model.plot_weight_distribution()

# Distribusi gradient di tiap layer
model.plot_gradient_distribution()

# contoh hasil prediksi 20 data:
# plt.figure(figsize=(15, 5))
# for i in range(20):
#     plt.subplot(2, 10, i+1)
#     plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
#     plt.title(f"True: {y_true_classes[i]}, Pred: {y_pred_classes[i]}")
#     plt.axis('off')
# plt.tight_layout()
# plt.show()

# Save and Load Model
Demonstrate saving the trained model to disk and loading it back, then verify the loaded model produces identical predictions.

In [ ]:
# Simpen model ke file
model.save("ffnn_model.pkl")

# Load model dari file
loaded_model = FFNN.load("ffnn_model.pkl")

# Cek apakah model yang di-load hasilin prediksi yang sama
y_pred_original = model.predict(X_test)
y_pred_loaded = loaded_model.predict(X_test)
identical_predictions = np.allclose(y_pred_original, y_pred_loaded)
print(f"Are predictions identical? {'Yes' if identical_predictions else 'No'}")

# Hitung akurasi untuk model yang di-load
y_pred_loaded_classes = np.argmax(y_pred_loaded, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
accuracy_loaded = np.mean(y_pred_loaded_classes == y_true_classes)
print(f"Test accuracy (loaded model): {accuracy_loaded:.4f}")